In [0]:
catalog_use = dbutils.widgets.get("catalog_use")
schema_use = dbutils.widgets.get("schema_use")

print(f"""
catalog_use: {catalog_use}
schema_use: {schema_use}
""")

In [0]:
%sql
DECLARE OR REPLACE VARIABLE catalog_use STRING DEFAULT :catalog_use;
DECLARE OR REPLACE VARIABLE schema_use STRING DEFAULT :schema_use;
SELECT catalog_use, schema_use;

In [0]:
%sql
USE IDENTIFIER(catalog_use || '.' || schema_use);
SELECT current_catalog(), current_schema();

In [0]:
import os
import yaml
from pathlib import Path

# Define the path to the metric views folder
metric_views_path = Path(os.path.abspath("../../fixtures/metric_views"))

# Find all .yml files
yml_files = list(metric_views_path.glob("*.yml"))

print(f"Found {len(yml_files)} YAML file(s):")
for yml_file in yml_files:
	print(f"  - {yml_file.name}")

In [0]:
# Loop through each YAML file and create metric views
for yml_file in yml_files:
	print(f"\nProcessing: {yml_file.name}")

	# Read the YAML file contents as-is
	with open(yml_file, 'r') as f:
		yaml_content = f.read()

	yaml_content = yaml_content.format(
		catalog = catalog_use,
		schema = schema_use
	)

	# Extract metric view name (remove .metric_view.yml extension)
	view_name = yml_file.stem.replace(".metric_view", "")

	# Build the fully qualified metric view name
	full_view_name = f"{catalog_use}.{schema_use}.{view_name}"

	print(f"  Creating/replacing metric view: {full_view_name}")
	
	# Build the CREATE OR REPLACE VIEW with YAML definition
	create_sql = f"""CREATE OR REPLACE VIEW {full_view_name}
WITH METRICS
LANGUAGE YAML
AS $$
{yaml_content}
$$"""
	
	print(f"  SQL to execute:\n{create_sql}\n")
	
	# Execute the SQL - this will create or replace the metric view
	spark.sql(create_sql)
	print(f"  ✓ Successfully created/replaced {full_view_name}")

print(f"\n{'='*60}")
print(f"Metric view creation complete!")